In [1]:
import torch
import pickle
import matplotlib.pyplot as plt
import numpy as np 
import statsmodels.api as sm
import networkx as nx
import seaborn as sns

from spikeometric.models import BernoulliGLM
from spikeometric.datasets import NormalGenerator, ConnectivityDataset
from spikeometric.stimulus import RegularStimulus

from torch_geometric.data import Data
from torch_geometric.utils import to_dense_adj, to_networkx, from_networkx
from CD_methods import SCM_learner

from tqdm import tqdm

In [2]:
network_data = torch.load('data/c_elegans_data.pt')

with open('data/c_elegans_spike_data_single_node_stimuli.pickle', 'rb') as f:
    spike_data = pickle.load(f)

In [17]:
n_neurons = network_data.num_nodes

G = to_networkx(network_data, node_attrs = ['position'])
position_dict = nx.get_node_attributes(G, 'position')

# sample neurons
n_obs = 90
index_obs = np.sort(np.random.choice(n_neurons, size = n_obs, replace = False))

stimulation_protocol = [[i] for i in index_obs]
spike_data_obs = dict()
spike_data_obs['null'] = spike_data['null'][index_obs]
for intervention in index_obs:
    spike_data_obs[str(intervention)] = spike_data[str(intervention)][index_obs]

In [20]:
G_obs = nx.subgraph(G, index_obs)
index_hidden = [node for node in range(n_neurons) if node not in index_obs]
confounders = []
for node in index_hidden:
    count = 0
    for _, v in G.out_edges(node):
        if v in index_obs:
            count += 1
    if count >= 2:
        confounders.append(node)
print('num. confounding variables = ',len(confounders))

num. confounding variables =  101


In [23]:
G_learned = SCM_learner(spike_data_obs, 
                        node_list=index_obs, 
                        stimulation_protocol=stimulation_protocol, 
                        alpha = 0.01)

100%|███████████████████████████████████████████| 90/90 [00:01<00:00, 74.39it/s]


In [24]:
G_true = nx.subgraph(G, index_obs)

A_true = nx.adjacency_matrix(G_true).todense() 
A_learned = nx.adjacency_matrix(G_learned).todense() 
A_diff = A_true - A_learned

SHD = np.sum(np.abs(A_true- A_learned))
TP = np.sum( (A_true == 1)*(A_learned==1) )
TN = np.sum( (A_true == 0)*(A_learned==0)) 
FP = np.sum(A_diff == -1)
FN = np.sum(A_diff == 1) 

index_hidden = [node for node in range(n_neurons) if node not in index_obs]
confounders = []
for node in index_hidden:
    count = 0
    for _, v in G.out_edges(node):
        if v in index_obs:
            count += 1
    if count >= 2:
        confounders.append(node)
print('num. confounding variables = ', len(confounders))

print('total edges (in true observed graph) = ',G_true.number_of_edges())
print('percentage of nodes observed = ', np.round(G_obs.number_of_nodes() / G.number_of_nodes() * 100, 2), '%')
print('SHD = ', SHD)
print('True positives = ', TP, ', Sensitivity = ', np.round(TP / (TP + FP) * 100, 2) )
print('True negatives = ', TN, ', Specificity = ', np.round(TN / (TN + FN)*100, 2) )
print('False positives = ', FP )
print('False negatives = ', FN )

num. confounding variables =  101
total edges (in true observed graph) =  190
percentage of nodes observed =  32.26 %
SHD =  0
True positives =  190 , Sensitivity =  100.0
True negatives =  7910 , Specificity =  100.0
False positives =  0
False negatives =  0


In [31]:
for n_obs in np.arange(10, 279, 10):
    print('n_obs = ', n_obs)
    index_obs = np.sort(np.random.choice(n_neurons, size = n_obs, replace = False))
    stimulation_protocol = [[i] for i in index_obs]
    spike_data_obs = dict()
    spike_data_obs['null'] = spike_data['null'][index_obs]
    for intervention in index_obs:
        spike_data_obs[str(intervention)] = spike_data[str(intervention)][index_obs]
    
    # count num. confounders
    G_obs = nx.subgraph(G, index_obs)
    index_hidden = [node for node in range(n_neurons) if node not in index_obs]
    confounders = []
    for node in index_hidden:
        count = 0
        for _, v in G.out_edges(node):
            if v in index_obs:
                count += 1
        if count >= 2:
            confounders.append(node)
    print('num. confounding variables = ',len(confounders))
    
    G_learned = SCM_learner(spike_data_obs, 
                        node_list=index_obs, 
                        stimulation_protocol=stimulation_protocol, 
                        alpha = 0.01)
    
    G_true = nx.subgraph(G, index_obs)

    A_true = nx.adjacency_matrix(G_true).todense() 
    A_learned = nx.adjacency_matrix(G_learned).todense() 
    A_diff = A_true - A_learned

    SHD = np.sum(np.abs(A_true- A_learned))
    TP = np.sum( (A_true == 1)*(A_learned==1) )
    TN = np.sum( (A_true == 0)*(A_learned==0)) 
    FP = np.sum(A_diff == -1)
    FN = np.sum(A_diff == 1) 

    print('percentage of nodes observed = ', np.round(G_obs.number_of_nodes() / G.number_of_nodes() * 100, 2), '%')
    print('SHD = ', SHD)
    print('True positives = ', TP, ', Sensitivity = ', np.round(TP / (TP + FP) * 100, 2) )
    print('True negatives = ', TN, ', Specificity = ', np.round(TN / (TN + FN)*100, 2) )
    print('False positives = ', FP )
    print('False negatives = ', FN )
    print('')

n_obs =  10
num. confounding variables =  16


100%|█████████████████████████████████████████| 10/10 [00:00<00:00, 3222.92it/s]

percentage of nodes observed =  3.58 %
SHD =  2
True positives =  0 , Sensitivity =  0.0
True negatives =  98 , Specificity =  98.99
False positives =  1
False negatives =  1

n_obs =  20
num. confounding variables =  43



100%|██████████████████████████████████████████| 20/20 [00:00<00:00, 346.48it/s]


percentage of nodes observed =  7.17 %
SHD =  28
True positives =  0 , Sensitivity =  0.0
True negatives =  372 , Specificity =  96.37
False positives =  14
False negatives =  14

n_obs =  30
num. confounding variables =  59


...

100%|█████████████████████████████████████████| 230/230 [00:24<00:00,  9.48it/s]


percentage of nodes observed =  82.44 %
SHD =  6
True positives =  1484 , Sensitivity =  99.66
True negatives =  51410 , Specificity =  100.0
False positives =  5
False negatives =  1

n_obs =  240
num. confounding variables =  35


100%|█████████████████████████████████████████| 240/240 [00:23<00:00, 10.11it/s]


percentage of nodes observed =  86.02 %
SHD =  11
True positives =  1545 , Sensitivity =  99.36
True negatives =  56044 , Specificity =  100.0
False positives =  10
False negatives =  1

n_obs =  250
num. confounding variables =  24


KeyboardInterrupt: 

### Notes
- Seems to be a 'critical threshold' for causal discovery, where the learned graph is reliable and accurate whenever we observe more than 28 % of the network 
 - Why this happens is unclear. But somehow the signal of the observed neurons is not strong enough with small samples. 
     Could it be:
     - Some set of 'critical nodes' are almost always included in the sampled data, fx high degree nodes
     - The number of confounders relative to observed nodes is reduced
- Need to investigate: 
    - The exact number of nodes required for threshold to be reached
    - Maybe look at the out- and in-degree distribution of nodes observed

In [62]:
sample_space = list(np.arange(279))
sample_space.remove(0)
sample_space.remove(6)
sample_space.remove(12)
sample_space.remove(98)
sample_space.remove(111)
sample_space.remove(129)
sample_space.remove(134)
sample_space.remove(142)
sample_space.remove(230)
sample_space.remove(238)
sample_space.remove(143)
sample_space.remove(146)
sample_space.remove(188)
sample_space.remove(270)

print(len(sample_space))

267


In [71]:
for n_obs in np.arange(70, 80, 2):
    index_obs = list(np.random.choice(128, size = n_obs, replace = False))
    index_obs += [129]
    index_obs=np.sort(index_obs)
    print('n_obs = ', len(index_obs))
    
    stimulation_protocol = [[i] for i in index_obs]
    spike_data_obs = dict()
    spike_data_obs['null'] = spike_data['null'][index_obs]
    for intervention in index_obs:
        spike_data_obs[str(intervention)] = spike_data[str(intervention)][index_obs]
    
    # count num. confounders
    G_obs = nx.subgraph(G, index_obs)
    index_hidden = [node for node in range(n_neurons) if node not in index_obs]
    confounders = []
    for node in index_hidden:
        count = 0
        for _, v in G.out_edges(node):
            if v in index_obs:
                count += 1
        if count >= 2:
            confounders.append(node)
    print('num. confounding variables = ',len(confounders))
    
    G_learned = SCM_learner(spike_data_obs, 
                        node_list=index_obs, 
                        stimulation_protocol=stimulation_protocol, 
                        alpha = 0.01)
    
    G_true = nx.subgraph(G, index_obs)

    A_true = nx.adjacency_matrix(G_true).todense() 
    A_learned = nx.adjacency_matrix(G_learned).todense() 
    A_diff = A_true - A_learned

    SHD = np.sum(np.abs(A_true- A_learned))
    TP = np.sum( (A_true == 1)*(A_learned==1) )
    TN = np.sum( (A_true == 0)*(A_learned==0)) 
    FP = np.sum(A_diff == -1)
    FN = np.sum(A_diff == 1) 

    print('percentage of nodes observed = ', np.round(G_obs.number_of_nodes() / G.number_of_nodes() * 100, 2), '%')
    print('SHD = ', SHD)
    print('True positives = ', TP, ', Sensitivity = ', np.round(TP / (TP + FP) * 100, 2) )
    print('True negatives = ', TN, ', Specificity = ', np.round(TN / (TN + FN)*100, 2) )
    print('False positives = ', FP )
    print('False negatives = ', FN )
    print('')

n_obs =  71
num. confounding variables =  103


100%|███████████████████████████████████████████| 71/71 [00:00<00:00, 80.70it/s]


percentage of nodes observed =  25.45 %
SHD =  270
True positives =  5 , Sensitivity =  3.57
True negatives =  4766 , Specificity =  97.25
False positives =  135
False negatives =  135

n_obs =  73
num. confounding variables =  84


100%|███████████████████████████████████████████| 73/73 [00:00<00:00, 87.02it/s]


percentage of nodes observed =  26.16 %
SHD =  284
True positives =  0 , Sensitivity =  0.0
True negatives =  5045 , Specificity =  97.26
False positives =  142
False negatives =  142

n_obs =  75
num. confounding variables =  95


...

percentage of nodes observed =  26.88 %
SHD =  296
True positives =  5 , Sensitivity =  3.27
True negatives =  5324 , Specificity =  97.3
False positives =  148
False negatives =  148

n_obs =  77
num. confounding variables =  99


100%|███████████████████████████████████████████| 77/77 [00:00<00:00, 79.04it/s]


percentage of nodes observed =  27.6 %
SHD =  0
True positives =  152 , Sensitivity =  100.0
True negatives =  5777 , Specificity =  100.0
False positives =  0
False negatives =  0

n_obs =  79
num. confounding variables =  97


100%|███████████████████████████████████████████| 79/79 [00:00<00:00, 79.46it/s]

percentage of nodes observed =  28.32 %
SHD =  0
True positives =  177 , Sensitivity =  100.0
True negatives =  6064 , Specificity =  100.0
False positives =  0
False negatives =  0



# Notes
- when high degree nodes are included in dataset, the learner struggles below 28% threshold. if not included, it still works. might be other 'critical nodes' to consider, like 0 and 6. Node 129 seems to be a problem.
- removing all nodes with in_degree > 20 from observed data is also bad...
- This info can be useful wrt designing experiments! see how many experiments are needed to get good results when targeting high degree nodes first.